In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import datetime

In [4]:
import bar_chart_race as bcr

In [5]:
df_labels = pd.read_excel("./Lista_105_Colombia_CIE9-y-CIE10.xls")

In [6]:
df_labels = df_labels[["No. Lista","Causa"]]

In [7]:
df = pd.read_csv("./nofetal2018.csv", sep=";")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3050: DtypeWarning: Columns (9,23,38) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
df = df.loc[df["MES"]<=8]

In [9]:
df["date"] = df["MES"].apply(lambda x:datetime.datetime(2020,x,1))

In [10]:
df = pd.merge(df,df_labels,left_on="CAU_HOMOL",right_on="No. Lista")

In [11]:
df["CAU_HOMOL"] = df["Causa"]

In [12]:
def fix_cancer(x):
    if "benigno" in x.lower():
        return "Otros tumores"
    if "tumor" in x.lower():
        return "Tumores malignos"
    else:
        return x

In [13]:
df["CAU_HOMOL"] = df["CAU_HOMOL"].apply(fix_cancer)

In [14]:
df = df.loc[df["CAU_HOMOL"].isin(list(df["CAU_HOMOL"].value_counts().head(19).index))]

In [15]:
df_cum = df.groupby(["CAU_HOMOL","date"]).agg("count")["COD_DPTO"].unstack().transpose().cumsum()

In [16]:
df_covid = pd.read_csv("https://www.datos.gov.co/api/views/gt2j-8ykr/rows.csv")

In [17]:
df_covid = df_covid[df_covid["Fecha de muerte"]==df_covid["Fecha de muerte"]]

In [18]:
df_covid["MES"] = df_covid["fecha reporte web"].apply(lambda x:int(x.split("-")[1]))
df_covid["DIA"] = df_covid["fecha reporte web"].apply(lambda x:int(x.split("-")[2][:2]))

In [19]:
df_covid["date"] = df_covid.apply(lambda x:datetime.datetime(2020,x["MES"],x["DIA"]),axis=1)

In [20]:
df_covid = df_covid["date"].value_counts().sort_index().cumsum().to_frame().rename(columns={"date":"Covid-19"})

In [21]:
df_cum = df_cum.join(df_covid, how="outer")

In [22]:
df_cum = df_cum.interpolate().fillna(0)

In [23]:
list(df_cum.columns)

['Accidentes de transporte de motor y secuelas ',
 'Agresiones (homicidios) y secuelas ',
 'Diabetes mellitus ',
 'Enfermedad por el VIH (SIDA) ',
 'Enfermedades cerebrovasculares ',
 'Enfermedades crónicas de las vías respiratorias inferiores ',
 'Enfermedades del hígado ',
 'Enfermedades hipertensivas ',
 'Enfermedades isquémicas del corazón ',
 'Insuficiencia renal ',
 'Lesiones autoinfligidas intencionalmente (suicidios) y secuelas ',
 'Neumonía ',
 'Otros tumores',
 'Signos, síntomas y afecciones mal definidas ',
 'Todas las demás enfermedades del sistema nervioso ',
 'Todas las demás enfermedades del sistema respiratorio ',
 'Todas las demás enfermedades del sistema urinario',
 'Todas las demás formas de enfermedad del corazón ',
 'Tumores malignos',
 'Covid-19']

In [24]:
names = {'Accidentes de transporte de motor y secuelas ':'Accidentes de tránsito',
 'Agresiones (homicidios) y secuelas ':'Homicidios',
 'Cáncer':'Cáncer',
 'Diabetes mellitus ':'Diabetes',
 'Enfermedad por el VIH (SIDA) ':'VIH (SIDA)',
 'Enfermedades cerebrovasculares ':'E. cerebrovasculares',
 'Enfermedades crónicas de las vías respiratorias inferiores ':'E. vías resp. inferiores',
 'Enfermedades del hígado ':'E. hígado',
 'Enfermedades del sistema osteomuscular y del tejido conjuntivo ':'E. sist. osteomuscular',
 'Enfermedades hipertensivas ':'E. hipertensivas',
 'Enfermedades isquémicas del corazón ':'E. isquémicas del corazón',
 'Insuficiencia renal ':'Ins. renal',
 'Lesiones autoinfligidas intencionalmente (suicidios) y secuelas ':'Suicidios',
 'Neumonía ':'Neumonía',
 'Signos, síntomas y afecciones mal definidas ':'Afecciones mal definidas',
 'Todas las demás enfermedades del sistema nervioso ':'E. sist. nervioso',
 'Todas las demás enfermedades del sistema respiratorio ':'E. sist. respiratorio',
 'Todas las demás enfermedades del sistema urinario':'E. sist. urinario',
 'Todas las demás formas de enfermedad del corazón ':'E. del corazón',
 'Covid-19':'** Covid-19 **'}

In [25]:
df_cum = df_cum.rename(columns=names)

In [28]:
df_cum

,Accidentes de tránsito,Homicidios,Diabetes,VIH (SIDA),E. cerebrovasculares,E. vías resp. inferiores,E. hígado,E. hipertensivas,E. isquémicas del corazón,Ins. renal,Suicidios,Neumonía,Otros tumores,Afecciones mal definidas,E. sist. nervioso,E. sist. respiratorio,E. sist. urinario,E. del corazón,Tumores malignos,** Covid-19 **
2020-01-01,629.000000,1138.000000,733.000000,251.000000,1323.000000,1242.000000,270.000000,834.000000,3585.000000,356.000000,253.000000,786.000000,305.000000,312.000000,266.000000,201.000000,189.0,354.000000,3527.000000,0.0
2020-02-01,1192.000000,2074.000000,1336.000000,469.000000,2535.000000,2201.000000,526.000000,1555.000000,6596.000000,648.000000,438.000000,1432.000000,561.000000,589.000000,508.000000,379.000000,357.0,697.000000,6609.000000,0.0
2020-03-01,1771.000000,3196.000000,1961.000000,709.000000,3871.000000,3297.000000,801.000000,2359.000000,9967.000000,977.000000,676.000000,2165.000000,873.000000,919.000000,799.000000,581.000000,560.0,1080.000000,10170.000000,0.0
2020-03-16,1808.500000,3274.285714,2006.642857,723.357143,3961.357143,3375.357143,818.857143,2406.857143,10195.214286,1001.142857,693.428571,2215.428571,892.000000,939.928571,817.357143,595.785714,575.5,1105.357143,10412.357143,1.0
2020-03-20,1846.000000,3352.571429,2052.285714,737.714286,4051.714286,3453.714286,836.714286,2454.714286,10423.428571,1025.285714,710.857143,2265.857143,911.000000,960.857143,835.714286,610.571429,591.0,1130.714286,10654.714286,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-03,4191.857143,7639.714286,4923.714286,1655.857143,9673.285714,8719.857143,1938.857143,5581.428571,24583.857143,2540.000000,1726.571429,6098.428571,2077.714286,2252.714286,2025.285714,1696.285714,1480.0,2744.285714,25217.714286,4242.0
2020-07-04,4274.285714,7794.571429,5019.571429,1687.285714,9876.428571,8887.285714,1978.285714,5692.142857,25057.285714,2589.000000,1760.857143,6218.142857,2117.571429,2295.571429,2067.428571,1726.428571,1509.0,2797.428571,25727.571429,4318.0
2020-07-05,4356.714286,7949.428571,5115.428571,1718.714286,10079.571429,9054.714286,2017.714286,5802.857143,25530.714286,2638.000000,1795.142857,6337.857143,2157.428571,2338.428571,2109.571429,1756.571429,1538.0,2850.571429,26237.428571,4396.0
2020-07-06,4439.142857,8104.285714,5211.285714,1750.142857,10282.714286,9222.142857,2057.142857,5913.571429,26004.142857,2687.000000,1829.428571,6457.571429,2197.285714,2381.285714,2151.714286,1786.714286,1567.0,2903.714286,26747.285714,4450.0


In [ ]:
bcr.bar_chart_race(df=df_cum.iloc[:-1], period_length=500)